In [1]:
import numpy as np
import scipy

In [35]:
def get_projection(q, x):
    # solve min |y-x|_2^2, s.t sum(y) = q, y >= 0
    q = np.float(q)
    if q == 0:
        return np.zeros(x.shape)
    n = len(x)
    min_x = np.min(x)
    x_sum = np.sum(x)
    sorted_x = np.sort(x)
    for i in range(n-1):
        lambda_2 = ((x_sum - np.sum(sorted_x[:i])) - q) / np.float(n - i)
        if lambda_2 <= min_x:
            if i == 0:
                thres = -np.inf
            else:
                thres = sorted_x[i-1]
            res = (x - lambda_2)
            res[x <= thres] = 0
            return res
    return q / np.sum(x) * x

In [229]:
get_projection(5, np.random.rand(20))

array([0.52763182, 0.        , 0.38565815, 0.0565248 , 0.28274823,
       0.        , 0.5784813 , 0.        , 0.        , 0.13740064,
       0.60825819, 0.02520689, 0.02380493, 0.12863033, 0.29770959,
       0.5333808 , 0.16338294, 0.61738011, 0.04822385, 0.58557744])

In [6]:
from cvxopt import matrix, solvers

In [7]:
def solve_cvx(q, x):
    q = np.float(q)
    n = len(x)
    Q = matrix(np.eye(n), (n,n))
    p = matrix(-2 * x, (n, 1))
    G = matrix(-np.eye(n), (n, n))
    h = matrix(np.zeros(n), (n, 1))
    A = matrix(np.ones(n), (1, n))
    b = matrix(q)
    sol = solvers.qp(Q, p, G, h, A, b)
    return np.array(sol['x']).flatten()

In [47]:
tmp_x = np.random.rand(600) * 2 - 10
target = 4
x1 = solve_cvx(target, tmp_x)
x2 = get_projection(target, tmp_x)

     pcost       dcost       gap    pres   dres
 0: -2.4183e+02 -4.0597e+01  2e+03  5e+01  1e-01
 1: -2.7854e+01  5.3752e+01  1e+02  4e+00  9e-03
 2:  6.4528e+01  6.2220e+01  4e+00  3e-02  7e-05
 3:  6.4791e+01  6.4327e+01  7e-01  5e-03  1e-05
 4:  6.4618e+01  6.4529e+01  1e-01  3e-04  6e-07
 5:  6.4550e+01  6.4545e+01  5e-03  8e-06  2e-08
 6:  6.4546e+01  6.4546e+01  2e-04  1e-07  3e-10
 7:  6.4546e+01  6.4546e+01  6e-06  1e-09  3e-12
Optimal solution found.


In [48]:
print np.sum(x1)
print np.sum(x2)

3.9999999999999987
3.999999999999037


In [49]:
print np.linalg.norm(x1 - tmp_x)
print np.linalg.norm(x2 - tmp_x)

220.87562014046455
220.89199806429198
